In [2]:
import numpy as np
# import google or-tools for solving comvinatorial optimization problem
import collections
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model

## Solving Combinatorial Optimization using Google OR-Tools

### Google OR-Tools example: Travelling Salesman Problem

The travelling salesman problem is a popular routing problem that can be solve by combinatorial optimization. Suppose a salesman is trying to go to n number of cities and the distance to travel from each city ot the other is denoted by $d_{ij}$. The goal is to find the minimum distance for the salesman to go to all cities. 

**Parameters:**
- number of cities: n 
- $d_{ij}$ : distance from city i to city j

**Decision variables:**
- $x_{ij} = 1$ if vehicle travels from node i to node j

**Dummy variable**
- $u_{i}$ : keeps the sequence of city i visited

**Formulation:**

Objective function:

we minimize the distance that the salesman have to go
$$\text{min} \sum_{i \in V} \sum_{j \in V} c_{ij}x_{ij}$$

subject to:

Each city needs to be visited: 
$$\sum_{i = 1}^n \sum_{i = 1, i \neq j} x_{ij} = 1 \quad j = 1, \dots, n$$ 
$$\sum_{j = 1}^n \sum_{j = 1, j \neq i} x_{ij} = 1 \quad i = 1, \dots, n$$

The cities need to be visited exactly once so there should not be a loop that visits a subset of cities without visiting all cities exactly once. This means $u_i < u_j$, so $u_j \geq u_i + 1$ if $x_{ij} = 1$, so city i is visited first before city j.

$$u_i - u_j + 1 \leq (n-1)(1-x_{ij}) \quad 2 \leq i \neq j \leq n$$
When $x_{ij} = 0$, the salesman doesn't visit city i to city j, so $u_i - u_j + 1 \leq (n-1)$, which implies that the biggest difference, if city i is visited before city j, is at most n-1. Since we need $u_i < u_j$, then $u_j$ is visited n-1 cities after $u_i$.

Since the starting point is 1, then the order of city i start from 2 to n, the number of cities.
$$2 \leq u_i \leq n$$


source: https://en.wikipedia.org/wiki/Travelling_salesman_problem (MTZ formulation)

The original CP solver is the foundation of the routing library.

### Google OR-Tools example: Scheduling Problem

The equipment scheduling problem is taken from Vanderbei (P. 389-390). Suppose there are n number of routes and m number of legs, a flight departing and arriving at some location. The goal is to find the minimum cost from the collection of potential routes. 

**parameters:**
- $c_j$ : cost of using route j
- $a_{ij} = 1$ of leg i is part of route j, and 0 otherwise

**decision variables:**
- $x_j = 1$ if route j is selected, and 0 otherwise

**Formulation:**

Objective function:

minimize the cost of the total route 
$$\text{min } \sum_{j = 1}^n c_j x_j$$

subject to:
$$\sum_{j = 1}^n a_{ij}x_j = 1 \quad i = 1, 2, \dots, m$$
$$x_j \in \{0,1\} j = 1, 2, \dots, n$$


Let there be 10 routes, and 15 leg.

In [60]:
n = 10
m = 10

c = [np.random.randint(1, 100) for j in range(n)] # cost of each route j
print(c)

a = [[np.random.randint(0, 2) for _ in range(n)] for _ in range(m)]
print(a)

[17, 91, 39, 57, 16, 83, 58, 42, 62, 16]
[[1, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 1, 0, 1, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 0, 1, 1, 1, 0], [1, 0, 0, 0, 1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 1, 1, 0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 1, 0, 0, 0, 1, 0, 0, 1]]


In [63]:
# create models, variables, and constraints
model = cp_model.CpModel()

# create decision variables
x = [model.NewBoolVar(f'x-{j}') for j in range(m)]

# define the objective function
# minimzing the total sum of cost of route j
objective = []
for j in range(n):
    objective.append(c[j] * x[j])
model.Minimize(sum(objective))

# constraint: 
# the sum of each leg given that it is part of the route is 1
# so, there can only be one route per leg
for i in range(m):
    model.Add(sum(a[i][j] * x[j] for j in range(n)) == 1)

# solve the model and evaluating solutions
solver = cp_model.CpSolver()
status = solver.Solve(model)

### Google OR-Tools example: Packing problem 

The knapsack problem is similar to the assignment problem, but we are trying to put as much valuable items as possible to a bag. The problem is not being able to fit everything, so we have to assign which item is more valuable to put into the bag. 

**parameters:**
- number of items: n
- $v_i$ : value of each item i
- $w_i$ : the weight of each item i
- c: maximum capacity of the bag

**decision variables:**
- $x_i = 1$ if we put item i into the bag, and 0 otherwise

**Formulation:**

Objective function:

maximize the total value of the items that goes into the bag
$$ \sum_{i = 1}^n v_i x_i$$
subject to:
$$ \sum_{i = 1}^n w_i x_i \leq c$$
The sum of all item placed into the bag should be less than the maximum capacity of the bag.


source: https://en.wikipedia.org/wiki/Knapsack_problem

Let there be n = 100 number of items. First, we create the dataset for 100 items. Both values and weights will be randomly generated and the maximum capacity of the bag is 200. 

In [3]:
values = [np.random.randint(1, 100) for i in range(100)]
weights = [np.random.randint(1, 100) for i in range(100)]
c = 200

In [4]:
# create models, variables, and constraints
model = cp_model.CpModel()

# create decision variable
# creates a 0-1 variable 
# x[i] = 1 if item i is in the bag
x = [model.NewBoolVar(f'x-{i}') for i in range(100)]

# define the objective function
# Maximize the total value of the items in the bag
objective = []
for i in range(100):
    objective.append(values[i] *  x[i])
model.Maximize(sum(objective))

# Define the constraitns
# The total weight of items into the bag shouldn't exceed its capacity
total_weights =[]
for i in range(100):
    total_weights.append(weights[i] * x[i])
model.Add(sum(total_weights) <= c)

# solve the model and evaluating solutions
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [5]:
# Print results
items_in_bag = []
# get the items in the bag from the solution of the solver
for i in range(100):
    if solver.Value(x[i]):
        items_in_bag.append(i)
        print('Item :', i, 'Value :', values[i])
# for all items in bag, get the total weight
total_weights = sum(weights[i] for i in items_in_bag)
# for all items in bag, get the total value
total_value = sum(values[i] for i in items_in_bag)
print('Total weight: ', total_weights)
print('Total value: ', total_value)

Item : 1 Value : 27
Item : 2 Value : 57
Item : 12 Value : 25
Item : 31 Value : 24
Item : 34 Value : 60
Item : 47 Value : 78
Item : 48 Value : 48
Item : 49 Value : 88
Item : 56 Value : 77
Item : 64 Value : 85
Item : 69 Value : 73
Item : 70 Value : 90
Item : 76 Value : 67
Item : 98 Value : 82
Total weight:  200
Total value:  881
